In [6]:
!pip install pyannote.audio


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install torchvision


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [27]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import split_audio_files

In [9]:
import requests
import os
import subprocess

def download_gh_file(url, local_filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_filename.strip(), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded {local_filename.strip()}")
    else:
        print(f"Failed to download {local_filename.strip()} from {url}. Status code: {response.status_code}")


In [10]:
def extract_audio(video_file, audio_file):
    command = [
        "ffmpeg",
        "-i", video_file,
        "-q:a", "0",
        "-map", "a",
        audio_file
    ]
    try:
        subprocess.run(command, check=True)
        print(f"Extracted audio to {audio_file}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to extract audio from {video_file}. Error: {e}")


In [11]:
# Ensure directories exist
os.makedirs('video_from_gh', exist_ok=True)
os.makedirs('audio_from_gh', exist_ok=True)

# Example usage:
from_id = 1 
to_id = 2

df = read_spreadsheet(sheet_id="1g6RVrHXygLzg4d2IduliUpg8fzvJzqYcbddS02nvyv8")

for index, row in df.iterrows():
    if not isinstance(row['File Name'], str) or not isinstance(row['asset_url'], str):
        break
    id = row['File Name']
    url_path = row['asset_url']
    video_filename = f"video_from_gh/{id}.MP4".strip()
    audio_filename = f"audio_from_gh/{id}.wav".strip()
    sr_no = row.iloc[0]
    if sr_no >= from_id and sr_no <= to_id:
        if os.path.exists(audio_filename):
            print(f"Audio file {audio_filename} already exists. Skipping extraction.")
            continue
        if not os.path.exists(video_filename):
            download_gh_file(url_path, video_filename)
        extract_audio(video_filename, audio_filename)

Downloaded video_from_gh/STT_NW0508.MP4


ffmpeg version 7.0.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

Extracted audio to audio_from_gh/STT_NW0508.wav
Downloaded video_from_gh/STT_NW0509.MP4


ffmpeg version 7.0.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

Extracted audio to audio_from_gh/STT_NW0509.wav


[out#0/wav @ 0x7da7180] video:0KiB audio:47832KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.000253%
size=   47832KiB time=00:04:37.66 bitrate=1411.2kbits/s speed=1.49e+03x    


In [16]:
!pip install pydub


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import subprocess
import os

def convert_to_16K(input_file, output_file):
    # Check if the output file already exists
    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping conversion.")
        return False
    
    # Construct the ffmpeg command
    ffmpeg_command = [
        "ffmpeg",
        "-i", input_file,
        "-f", "wav",            # Force input format as WAV
        "-bitexact",            # Preserve exact precision
        "-acodec", "pcm_s16le", # Audio codec: PCM signed 16-bit little-endian
        "-ac", "1",             # Mono channel
        "-ar", "16000",         # 16 kHz sample rate
        output_file,
        "-y"                    # Overwrite output file if it exists
    ]
    
    # Join the ffmpeg command into a single string for subprocess
    ffmpeg_command_str = " ".join(ffmpeg_command)
    
    try:
        # Run ffmpeg command using subprocess
        subprocess.run(ffmpeg_command_str, shell=True, check=True, capture_output=True)
        print(f"Conversion successful: {input_file} -> {output_file}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Conversion failed: {e}")
        return False


In [18]:
from pathlib import Path
path = Path('./audio_from_gh/')
full_audios = set([e.stem for e in path.iterdir() if e.is_file()])

In [19]:
print(full_audios)

{'STT_NW0508', 'STT_NW0509'}


In [24]:
os.makedirs('audio_16k', exist_ok=True)
for full_audio in full_audios:
    print(full_audio)
    input_file = f"audio_from_gh/{full_audio}.wav"
    output_file = f"audio_16k/{full_audio}.wav"
    convert_to_16K(input_file, output_file)

STT_NW0508
Output file audio_16k/STT_NW0508.wav already exists. Skipping conversion.
STT_NW0509
Output file audio_16k/STT_NW0509.wav already exists. Skipping conversion.


In [25]:
import shutil
import os
for file in os.listdir("audio_16k"):
    if not os.path.exists( os.path.join("./full_audio/",file)):
        os.makedirs("./full_audio/", exist_ok=True)
        print("coppying ", file)
        shutil.copy(os.path.join("audio_16k",file), os.path.join("./full_audio/",file))

In [28]:
split_audio_files('STT_NW', 'wav')

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
collect_segments('STT_NW', 'after_split', 'segments_nw')

In [ ]:
! aws s3 cp segments_nw  s3://monlam.ai.stt/wav16k/ --recursive